In [1]:
%matplotlib inline

import cv2 # The OpenCV library; install using `pip install opencv-contrib-python`
import numpy as np # Helpful when working with arrays; install using `pip install numpy`
from matplotlib import pyplot as plt # Good for graphing; install using `pip install matplotlib`
from matplotlib import image as image

In [10]:
import cv2
import numpy as np

# Video and background subtraction initialization
video = cv2.VideoCapture("Images/traffic.mp4")
sub_type = 'MOG2'
backSub = (cv2.createBackgroundSubtractorMOG2(varThreshold=16, detectShadows=True) 
           if sub_type == "MOG2" else 
           cv2.createBackgroundSubtractorKNN(dist2Threshold=1000, detectShadows=True))

kernel = np.ones((9, 9), dtype=np.uint8)

# Kalman filter tracking structures
tracked_objects = {}
next_object_id = 0
max_missed_frames = 10  # Max frames to keep an object if it’s not detected
min_appearance_frames = 3  # Minimum frames for an ID to be shown

def create_kalman_filter():
    kf = cv2.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    kf.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
    return kf

def initialize_kalman_for_object(center):
    global next_object_id
    kf = create_kalman_filter()
    kf.statePre = np.array([center[0, 0], center[1, 0], 0, 0], dtype=np.float32)
    kf.correct(center)
    tracked_objects[next_object_id] = {
        'kf': kf,
        'id': next_object_id,
        'missed_frames': 0,
        'appearance_count': 0  # Tracks how many consecutive frames the object has appeared
    }
    next_object_id += 1

def update_tracked_objects(detected_centers):
    global tracked_objects
    unmatched_detections = detected_centers[:]
    for obj_id, data in list(tracked_objects.items()):
        kf = data['kf']
        predicted = kf.predict()
        obj_position = np.array([predicted[0], predicted[1]])

        # Find the closest center to this object
        distances = [np.linalg.norm(center.flatten() - obj_position) for center in unmatched_detections]
        if distances and min(distances) < 50:
            matched_index = np.argmin(distances)
            kf.correct(unmatched_detections[matched_index])
            data['missed_frames'] = 0
            data['appearance_count'] += 1  # Increase appearance count if detected
            unmatched_detections.pop(matched_index)
        else:
            # Increment missed frame count and reset appearance count
            data['missed_frames'] += 1
            data['appearance_count'] = 0

        # Remove if the object has been lost for too many frames
        if data['missed_frames'] > max_missed_frames:
            del tracked_objects[obj_id]

    # Initialize new objects for unmatched detections
    for center in unmatched_detections:
        initialize_kalman_for_object(center)

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Background subtraction and thresholding
    fg_mask = backSub.apply(frame)
    _, thresh = cv2.threshold(fg_mask, 0, 255, cv2.THRESH_BINARY)
    motion_mask = cv2.medianBlur(thresh, 3)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Detect contours
    contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_centers = [np.array([[x + w / 2], [y + h / 2]], dtype=np.float32) 
                        for contour in contours if cv2.contourArea(contour) > 500 
                        for x, y, w, h in [cv2.boundingRect(contour)]]
    
    # Update tracked objects based on detected centers
    update_tracked_objects(detected_centers)
    
    # Predict and draw tracked objects
    for obj_id, data in tracked_objects.items():
        kf = data['kf']
        predicted = kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        
        # Only display the ID if the appearance count exceeds the threshold
        if data['appearance_count'] >= min_appearance_frames:
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
            cv2.putText(frame, f'ID {data["id"]}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display frames
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
video.release()
cv2.destroyAllWindows()

